In [1]:
from tqdm.auto import tqdm 
import pandas as pd
import numpy as np
import xarray as xr
import netCDF4 as nf
from netCDF4 import Dataset
%matplotlib inline
import glob
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [2]:
datapath="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/Data/ECMWF/ERA5_25kmx3hr/"
path="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/besttracks/wp/"
output="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/create_ts/outputs/"
target="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/create_ts/outputs/targets/"

### Read tracks

In [102]:
track = sorted(glob.glob(path+'wp_2021.csv'))

In [103]:
tracksDF = pd.read_csv(track[0])

In [104]:
tracksDF['name'].unique()

array(['CHAMPI', 'IN-FA', 'LUPIT', 'OMAIS', 'CONSON', 'CHANTHU', 'RAI'],
      dtype=object)

### Extract time series

In [105]:
def largearea_withpres(dataset=None,invar=None,indices=None,tc_irad=None):
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","plev","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices))),
               plev=(["plev"],dataset.plev.data)),
    attrs=dict(description="coords with matrices"),)
    
    LATN,LATS,LONE,LONW = tc_irad[0,:]
    testsmall = ds['variable'][0,0,:,:].sel(lat=slice(LATS,LATN),lon=slice(LONE,LONW))
    if testsmall.shape[0]<testsmall.shape[1]:
        rgspt = int(testsmall.shape[0])
    else:
        rgspt = int(testsmall.shape[1])
    rgspt=16
    var_out=np.zeros((len(indices),len(dataset.plev.data),rgspt,rgspt))
    del testsmall
    
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_irad[it,:]
        for ip in range(len(dm1.plev.data)):
            try:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
            except:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:rgspt,0:rgspt]
    return var_out  

In [106]:
##########function############
def output_indices(TCtrack=None,ERA5date=None,ERA5hour=None):
    allindices = []
    for timeidx in range(len(TCtrack)):#len(track['time'])):
        datetrack,hourtrack = TCtrack['time'][timeidx].split(':')[0],TCtrack['time'][timeidx].split(':')[1][0:2]
        ####################################################################################################
        # Find the indices in ERA5 data with the same date as track
        ####################################################################################################
        dateind = []
        for ind,obj in enumerate(ERA5date):
            if obj==datetrack:
                dateind.append(ind)
        del ind,obj
        hourind = []
        hourextract = ERA5hour[int(np.min(np.asarray(dateind))):int(np.max(np.asarray(dateind)))+1]
        for ind,obj in enumerate(hourextract):
            if obj==hourtrack:            
                hourind.append(ind)
        allindices.append((int(np.min(np.asarray(dateind))),int(hourind[0])))
    return allindices

def extract_var(dataset=None,var='var138',indices=None):
    extractedvar = []
    for i in (range(len(indices))):
        realindex = indices[i][0]+indices[i][1]
        extractedvar.append(dataset[var][int(realindex),...].data)
    return np.asarray(extractedvar)

def smallarea(dataset=None,invar=None,indices=None,tc_irad=None):
    if len(invar.shape) != 3:
        invar = np.squeeze(invar)
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices)))),
    attrs=dict(description="coords with matrices"),)
    
    LATN,LATS,LONE,LONW = tc_irad[0,:]
    testsmall = ds['variable'][0,:,:].sel(lat=slice(LATS,LATN),lon=slice(LONE,LONW))
    if testsmall.shape[0]<testsmall.shape[1]:
        rgspt = int(testsmall.shape[0])
    else:
        rgspt = int(testsmall.shape[1])
    rgspt=16
    var_out=np.zeros((len(indices),rgspt,rgspt))
    del testsmall
    
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_irad[it,:]
        try:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
        except:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:rgspt,0:rgspt]
    return var_out

def largearea_withpres(dataset=None,invar=None,indices=None,tc_irad=None):
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","plev","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices))),
               plev=(["plev"],dataset.plev.data)),
    attrs=dict(description="coords with matrices"),)
    
    LATN,LATS,LONE,LONW = tc_irad[0,:]
    testsmall = ds['variable'][0,0,:,:].sel(lat=slice(LATS,LATN),lon=slice(LONE,LONW))
    if testsmall.shape[0]<testsmall.shape[1]:
        rgspt = int(testsmall.shape[0])
    else:
        rgspt = int(testsmall.shape[1])
    rgspt=16
    var_out=np.zeros((len(indices),len(dataset.plev.data),rgspt,rgspt))
    del testsmall
    
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_irad[it,:]
        for ip in range(len(dm1.plev.data)):
            try:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
            except:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:rgspt,0:rgspt]
    return var_out  

In [107]:
stormnames = list(tracksDF['name'].unique())

In [108]:
stormnames

['CHAMPI', 'IN-FA', 'LUPIT', 'OMAIS', 'CONSON', 'CHANTHU', 'RAI']

In [110]:
#for dm1 files functions
def make_timeseries_step1(TCname=None,era5_date=None,era5_hour=None):
    track=tracksDF[tracksDF['name']==TCname].reset_index()
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    lonx=np.mod(lon1,360)
    pos = arr = np.stack((lat1, lonx), axis=1)
    #pos = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    return pos,indices_store,tc_irad

def make_timeseries_step3(dm1=None,pvarname=None,pvardict=None,indices_store=None):
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    return ts_pdict

In [111]:
def readyear_automatic(year=None):
    dm2 = xr.open_dataset(datapath+'/slev_vars/radvars_'+str(year)+'.nc')
    #tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/besttracks/nio/*_'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,era5_date,era5_hour

dm2,era5_date,era5_hour = readyear_automatic(2021)

In [112]:
dm2

<xarray.Dataset>
Dimensions:  (time: 2920, lon: 1440, lat: 721)
Coordinates:
  * time     (time) datetime64[ns] 2021-01-01 ... 2021-12-31T21:00:00
  * lon      (lon) float64 0.0 0.25 0.5 0.75 1.0 ... 359.0 359.2 359.5 359.8
  * lat      (lat) float64 90.0 89.75 89.5 89.25 ... -89.25 -89.5 -89.75 -90.0
Data variables: (12/16)
    var59    (time, lat, lon) float32 ...
    var29    (time, lat, lon) float32 ...
    var232   (time, lat, lon) float32 ...
    var231   (time, lat, lon) float32 ...
    var58    (time, lat, lon) float32 ...
    var34    (time, lat, lon) float32 ...
    ...       ...
    var39    (time, lat, lon) float32 ...
    var54    (time, lat, lon) float32 ...
    var147   (time, lat, lon) float32 ...
    var146   (time, lat, lon) float32 ...
    var88    (time, lat, lon) float32 ...
    var137   (time, lat, lon) float32 ...
Attributes:
    CDI:          Climate Data Interface version 1.9.10 (https://mpimet.mpg.d...
    Conventions:  CF-1.6
    institution:  European Centre for Medium-Range Weather Forecasts
    history:      Wed Jul 06 08:40:00 2022: cdo -f nc copy radvars_2021.grib ...
    CDO:          Climate Data Operators version 1.9.10 (https://mpimet.mpg.d...

In [113]:
TCrl_ts = []
for TCname in tqdm(stormnames):
    ###########################################################################
    pos,indices_store,tc_irad = make_timeseries_step1(TCname=TCname,era5_date=era5_date,era5_hour=era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    smallsvarout = [smallarea(dm2,mysvar[i],indices_store,tc_irad) for i in (range(len(mysvar)))]
    svarname = ['cape','inst_10m_wnd_gst','inst_moisture_flux','inst_ssh_flux','surfmean_swr_flux','surfmean_lhf',\
           'surfmean_lwr_flux','surfmean_shf','dwnwrdmean_swr_flux','topmean_lwr_flux','topmean_swr_flux',\
           'vimean_moisture_div','surf_lhf','surf_shf','tot_suprcool_liqwtr','tot_wtr_vpr']
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    #############################################################################################

    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...].flatten() for i in range(len(indices_store))]
        tsdict[svarname[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCrl_ts.append(tsdict)

  0%|          | 0/7 [00:00<?, ?it/s]

In [114]:
storeTCdicts = {}
for ind in range(len(TCrl_ts)):
    storeTCdicts[stormnames[ind]] = pd.DataFrame.from_dict(TCrl_ts[ind])

In [115]:
for ind,obj in enumerate(stormnames):
    storeTCdicts[obj].to_csv(output+'2021_radvar_wpac_'+str(obj)+'.csv')

In [116]:
pd.DataFrame.from_dict(TCrl_ts[0])

,cape,inst_10m_wnd_gst,inst_moisture_flux,inst_ssh_flux,surfmean_swr_flux,surfmean_lhf,surfmean_lwr_flux,surfmean_shf,dwnwrdmean_swr_flux,topmean_lwr_flux,topmean_swr_flux,vimean_moisture_div,surf_lhf,surf_shf,tot_suprcool_liqwtr,tot_wtr_vpr
0,498.690430,8.516520,-0.000082,-24.094513,221.042053,-164.354126,-47.102600,-22.028931,1001.054199,-182.024689,629.229858,-0.000116,-591676.50,-79304.125,0.039280,55.525772
1,529.184570,7.963530,-0.000081,-25.950500,356.182617,-156.882874,-47.551315,-24.558838,1281.680908,-185.620483,908.424316,-0.000216,-564780.00,-88413.750,0.068474,55.072567
2,662.480469,7.962962,-0.000075,-24.823608,188.948059,-152.807373,-47.076202,-25.250793,888.274658,-199.627319,542.528931,-0.000249,-550107.25,-90903.250,0.112733,55.419228
3,823.904297,9.434483,-0.000067,-22.171570,7.745483,-172.003479,-46.964447,-21.026855,66.895630,-227.340454,38.192871,-0.000162,-619212.25,-75695.750,0.080411,55.437538
4,485.898438,9.505889,-0.000072,-17.934326,0.000000,-168.942200,-49.074371,-22.515198,0.000000,-237.425018,0.000000,-0.000197,-608192.25,-81054.250,0.068682,56.334419
5,574.767090,10.040740,-0.000073,-22.990723,0.000000,-178.633179,-47.171326,-26.780701,0.000000,-236.131653,0.000000,-0.000302,-643080.25,-96408.250,0.080338,55.731789
6,748.182129,9.958516,-0.000070,-27.421509,0.000000,-178.006714,-48.445984,-30.909973,0.000000,-219.893921,0.000000,-0.000422,-640823.75,-111278.250,0.116753,56.256897
7,757.934570,9.898953,-0.000075,-31.685059,10.781860,-182.394836,-49.255920,-27.611755,139.106201,-178.313354,67.293335,-0.000435,-656621.25,-99404.000,0.087824,56.597248
8,423.187500,10.356182,-0.000089,-33.242493,257.176941,-196.879333,-50.082092,-32.796997,955.859497,-192.021729,615.587158,-0.000564,-708766.50,-118067.750,0.152604,57.172981
9,618.954590,10.032948,-0.000091,-32.350647,403.886536,-202.113281,-49.794891,-33.826050,1294.192261,-203.930389,881.089600,-0.000641,-727609.50,-121772.375,0.222897,57.235367


In [117]:
def readyear_automatic(year=None):
    dm2 = xr.open_mfdataset([datapath+'/prate/prates_'+str(year)+'.nc',datapath+'/mslp/mslp_'+str(year)+'.nc',datapath+'/sst/sst_'+str(year)+'.nc'])
    #tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/besttracks/nio/*_'+str(year)+'*'))
    #era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    #era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2#,era5_date,era5_hour

dm2 = readyear_automatic(2021)

In [118]:
TCpr_ts = []
for TCname in tqdm(stormnames):
    ###########################################################################
    pos,indices_store,tc_irad = make_timeseries_step1(TCname=TCname,era5_date=era5_date,era5_hour=era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]

    ################################################################################################
    
    smallsvarout = [smallarea(dm2,mysvar[i],indices_store,tc_irad) for i in (range(len(mysvar)))]
    
    svarname = ['mn_mslp','conv_rrate','ls_rrate','mn_conv_prate','mn_ls_prate','mn_tot_prate','sst']
    
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    ##################################################################################################
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...].flatten() for i in range(len(indices_store))]
        tsdict[svarname[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCpr_ts.append(tsdict)

  0%|          | 0/7 [00:00<?, ?it/s]

In [119]:
storeTCdicts = {}
for ind in range(len(TCrl_ts)):
    storeTCdicts[stormnames[ind]] = pd.DataFrame.from_dict(TCpr_ts[ind])

In [120]:
for ind,obj in enumerate(stormnames):
    storeTCdicts[obj].to_csv(output+'2021_presstvar_wpac_'+str(obj)+'.csv')

In [121]:
pd.DataFrame.from_dict(TCpr_ts[3])

,mn_mslp,conv_rrate,ls_rrate,mn_conv_prate,mn_ls_prate,mn_tot_prate,sst
0,101300.527344,0.000319,0.000335,0.000258,0.000335,0.000593,301.700268
1,101171.506836,0.000227,0.000263,0.000247,0.000282,0.000529,301.750984
2,101200.511719,0.000304,0.000173,0.000262,0.000187,0.000449,301.789276
3,101351.300781,0.000312,0.000235,0.000347,0.000215,0.000563,301.803307
4,101318.576172,0.000218,0.000181,0.000225,0.000194,0.000419,301.821163
...,...,...,...,...,...,...,...
104,100396.659180,0.000156,0.000161,0.000221,0.000181,0.000402,300.144645
105,100379.177734,0.000149,0.000667,0.000174,0.000656,0.000830,299.928871
106,100230.510742,0.000293,0.000622,0.000259,0.000787,0.001046,299.401319
107,100195.346680,0.000173,0.000730,0.000210,0.000914,0.001124,298.677623


In [122]:
def readyear_automatic(year=None):
    dm2=xr.open_dataset(datapath+'/slev_vars/svars_'+str(year)+'.nc') #tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/besttracks/nio/*_'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,era5_date,era5_hour

dm2,era5_date,era5_hour = readyear_automatic(2021)

In [123]:
TCsl_ts = []
for TCname in tqdm(stormnames):
    ###########################################################################
    pos,indices_store,tc_irad = make_timeseries_step1(TCname=TCname,era5_date=era5_date,era5_hour=era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    ################################################################################################
    
    smallsvarout = [smallarea(dm2,mysvar[i],indices_store,tc_irad) for i in (range(len(mysvar)))]
    
    svarname = ['u10','v10','2mdewtmp','2mtmp','conv_ppt','tot_cld_ice',\
           'tot_cldwtr','tot_cld_rain','vi_div_cld_froz_wtr','vi_div_cld_liq_wtr','vi_div_gpot_flux',\
           'vi_div_ke_flux','vi_div_mass_flux','vi_div_moisture_flux','vi_div_olr_flux','vi_div_tot_enrgy_flux',\
           'vi_ke','vi_pe_inte','vi_pe_ie_latentenrgy','vi_temp','vi_olr','vi_tot_enrgy','vi_moisture_div']
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    ##################################################################################################
    #############################################################################################
    tsdict = {}
    for ind,obj in tqdm(enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...].flatten() for i in range(len(indices_store))]
        tsdict[svarname[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCsl_ts.append(tsdict)

  0%|          | 0/7 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [124]:
storeTCdicts = {}
for ind in range(len(TCrl_ts)):
    storeTCdicts[stormnames[ind]] = pd.DataFrame.from_dict(TCsl_ts[ind])

In [125]:
for ind,obj in enumerate(stormnames):
    storeTCdicts[obj].to_csv(output+'2021_slvars_wpac_'+str(obj)+'.csv')

In [126]:
#for dm1 files functions
def make_timeseries_step1(TCname=None,era5_date=None,era5_hour=None):
    track=tracksDF[tracksDF['name']==TCname].reset_index()
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    lonx=np.mod(lon1,360)
    pos = arr = np.stack((lat1, lonx), axis=1)
    #pos = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    return pos,indices_store,tc_irad

def make_timeseries_step3(dm1=None,pvarname=None,pvardict=None,indices_store=None):
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    return ts_pdict

In [127]:
dm1=xr.open_dataset(datapath+'/vor/vor_2021.nc')

In [128]:
#For vars with 21 vertical levels vorticity humidity file##################
TCv_ts = []

for TCname in tqdm(stormnames):
    ############################################################################
    pos,indices_store,tc_irad = make_timeseries_step1(TCname=TCname,era5_date=era5_date,era5_hour=era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store,tc_irad) for i in (range(len(mypvar)))]    
    pvarname = ['vor']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    #############################################################################################
    TCv_ts.append(ts_pdict)

  0%|          | 0/7 [00:00<?, ?it/s]

In [129]:
dm1=xr.open_dataset(datapath+'/rhum/rhum_2021.nc')

In [130]:
TCrh_ts = []

for TCname in tqdm(stormnames):
    ############################################################################
    pos,indices_store,tc_irad = make_timeseries_step1(TCname=TCname,era5_date=era5_date,era5_hour=era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store,tc_irad) for i in (range(len(mypvar)))]    
    pvarname = ['rhum']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    #############################################################################################
    TCrh_ts.append(ts_pdict)

  0%|          | 0/7 [00:00<?, ?it/s]

In [131]:
dm1=xr.open_dataset(datapath+'/pvor/pvor_2021.nc')

In [132]:
#potential vorticity
TCp2_ts = []

for TCname in tqdm(stormnames):
    ############################################################################
    pos,indices_store,tc_irad = make_timeseries_step1(TCname=TCname,era5_date=era5_date,era5_hour=era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store,tc_irad) for i in (range(len(mypvar)))]    
    pvarname = ['pvor']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCp2_ts.append(ts_pdict)

  0%|          | 0/7 [00:00<?, ?it/s]

In [133]:
dm1=xr.open_dataset(datapath+'/div/div_2021.nc')

In [134]:
#divergence
TCd_ts = []
for TCname in tqdm(stormnames):
    ############################################################################
    pos,indices_store,tc_irad = make_timeseries_step1(TCname=TCname,era5_date=era5_date,era5_hour=era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store,tc_irad) for i in (range(len(mypvar)))]    
    pvarname = ['div']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()

    #############################################################################################
    TCd_ts.append(ts_pdict)

  0%|          | 0/7 [00:00<?, ?it/s]

In [135]:
pilv=[str(int(obj)) for obj in dm1.plev.data/100]

In [136]:
pilv

['50',
 '100',
 '200',
 '250',
 '300',
 '400',
 '500',
 '600',
 '700',
 '800',
 '850',
 '925',
 '1000']

In [137]:
dm1=xr.open_dataset(datapath+'/gpot/gpot_2021.nc')

In [138]:
#geopotential
TCgp_ts = []
for TCname in tqdm(stormnames):
    ############################################################################
    pos,indices_store,tc_irad = make_timeseries_step1(TCname=TCname,era5_date=era5_date,era5_hour=era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store,tc_irad) for i in (range(len(mypvar)))]    
    pvarname = ['gpot']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCgp_ts.append(ts_pdict)

  0%|          | 0/7 [00:00<?, ?it/s]

In [139]:
dm1=xr.open_dataset(datapath+'/tlevs/tlevs_2021.nc')

In [140]:
#temperature levels
TCtl_ts = []
for TCname in tqdm(stormnames):
    ############################################################################
    pos,indices_store,tc_irad = make_timeseries_step1(TCname=TCname,era5_date=era5_date,era5_hour=era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store,tc_irad) for i in (range(len(mypvar)))]    
    pvarname = ['temp']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCtl_ts.append(ts_pdict)

  0%|          | 0/7 [00:00<?, ?it/s]

In [141]:
dm1=xr.open_dataset(datapath+'/vvel/w_wnd_2021.nc')

In [142]:
#Vertical velocity
TCvv_ts = []
for TCname in tqdm(stormnames):
    ###########################################################################
    pos,indices_store,tc_irad = make_timeseries_step1(TCname=TCname,era5_date=era5_date,era5_hour=era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store,tc_irad) for i in (range(len(mypvar)))]    
    pvarname = ['vvel']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    ts_pdict = make_timeseries_step3(dm1=dm1,pvarname=pvarname,pvardict=pvardict,indices_store=indices_store)
    #############################################################################################
    TCvv_ts.append(ts_pdict)

  0%|          | 0/7 [00:00<?, ?it/s]

In [143]:
plv = [str(int(obj)) for obj in dm1.plev.data/100]

In [144]:
dx=xr.open_dataset(datapath+'/tlevs/tlevs_2021.nc')

In [145]:
plev=[str(int(obj)) for obj in dx.plev.data/100]

In [146]:
plv

['10',
 '20',
 '30',
 '50',
 '70',
 '100',
 '150',
 '200',
 '250',
 '300',
 '400',
 '500',
 '600',
 '700',
 '850',
 '925',
 '975',
 '1000']

In [147]:
len(plv),len(pilv),len(plev),TCgp_ts[0]['gpot'].shape

(18, 13, 21, (57, 20))

In [148]:
def create_pandadict(divdict=None,vordict=None,rhdict=None,gpotdict=None,tempdict=None,vveldict=None,divlv=pilv,vorlv=plev,\
                     rhlev=plev,gpotlv=plev,templv=plev,vvellv=plv):
    tempdivdict = {'div_'+divlv[i]:divdict['div'][:,i] for i in range(divdict['div'].shape[1])}
    tempvordict = {'vor_'+vorlv[i]:vordict['vor'][:,i] for i in range(vordict['vor'].shape[1])}
    temprhdict = {'rh_'+rhlev[i]:rhdict['rhum'][:,i] for i in range(rhdict['rhum'].shape[1])}
    tempgpotdict = {'gpot_'+gpotlv[i]:gpotdict['gpot'][:,i] for i in range(gpotdict['gpot'].shape[1])}
    temptempdict = {'temp_'+templv[i]:tempdict['temp'][:,i] for i in range(tempdict['temp'].shape[1])}
    tempvveldict = {'vvel_'+vvellv[i]:vveldict['vvel'][:,i] for i in range(vveldict['vvel'].shape[1])}   
    dict1 = {**tempdivdict,**tempvordict} # change this later
    dict2 = {**temprhdict,**tempgpotdict}
    dict3 = {**temptempdict,**tempvveldict}
    combine1={**dict1,**dict2} # change this later
    alldict = {**combine1,**dict3} # change this later
    #alldict= {**dict2,**dict3}
    return alldict

In [149]:
storeTCdicts = {}
for ind in range(len(TCp2_ts)):
    storeTCdicts[stormnames[ind]] = pd.DataFrame.from_dict(create_pandadict(divdict=TCd_ts[ind],vordict=TCv_ts[ind],\
                                                                            rhdict=TCrh_ts[ind],gpotdict=TCgp_ts[ind],\
                                                                            tempdict=TCtl_ts[ind],vveldict=TCvv_ts[ind]))

In [150]:
for ind,obj in enumerate(stormnames):
    storeTCdicts[obj].to_csv(output+'2021_innerlevs_wpac_'+str(obj)+'.csv')

the end of script